# Create index for interpolated data

The appetizer creates one file per time step, that we want to open as one xarray dataset. This operation is VERY slow if you decode times, and still quite slow when you don't, and you have to recreate the time axis by hand afterwards. You can create an index, to make it looks like a fake zarr file for xarray, that will load instantly and even be faster to process.

Created by: 
Tobias Kölling (@d70-t)

In [1]:
import json
import glob

import tqdm
import kerchunk.hdf
import fsspec

In [2]:
path_to_output = "/work/ab0995/a270088/NextGems_public/appetizer/IFS2.5/2t/netcdf/"
files = "tco3999-ng5_pc_*.nc"

In [3]:
urls = list(sorted(glob.glob(f"{path_to_output}/{files}")))
singles = []
for u in tqdm.tqdm(urls):
    with fsspec.open(u) as inf:
        h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, u, inline_threshold=100)
        singles.append(h5chunks.translate())

100%|██████████| 985/985 [02:19<00:00,  7.05it/s]


This is a hack to replace the times within the single reference objects such that they use common units. That way, MultiZarrToZarr doesn't get confused.

In [7]:
import base64
import xarray as xr

def encode_value(v):
    try:
        return v.decode("ascii")
    except UnicodeDecodeError:
        return "base64:" + base64.b64encode(v).decode("ascii")
    
def fix_time(single):
    t = xr.open_dataset(
        "reference://", engine="zarr",
        backend_kwargs={
            "storage_options": {
                "fo": single,
            },
            "consolidated": False
        }
    )[["time"]].compute()
    t.time.encoding = {}
    m = {}
    t.to_zarr(m, encoding={"time": {"units": "seconds since 1990-01-01", "dtype": "i4", "compressor": None}})
    return {
        "version": single["version"],
        "templates": single["templates"],
        "refs": {
            **single["refs"],
            **{k: encode_value(v) for k, v in m.items() if k.startswith("time/")}
        }
    }

In [8]:
from kerchunk.combine import MultiZarrToZarr
mzz = MultiZarrToZarr(
    [fix_time(s) for s in singles],
    concat_dims=["time"]
)

out = mzz.translate()

In [9]:
import xarray as xr
ds = xr.open_dataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo": out,
        },
        "consolidated": False
    }
)

In [10]:
ds

<xarray.Dataset>
Dimensions:    (time: 985, lat: 2700, lon: 5400)
Coordinates:
  * lat        (lat) float64 -90.0 -89.93 -89.87 -89.8 ... 89.8 89.87 89.93 90.0
    latitude   (time, lat, lon) float64 ...
  * lon        (lon) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
    longitude  (time, lat, lon) float64 ...
  * time       (time) datetime64[ns] 2020-01-20 ... 2020-03-01
Data variables:
    2t         (time, lat, lon) float32 ...
Attributes:
    experiment_id:  tco3999-ng5
    interpolation:  nearest neighbour

In [12]:
files[:-5]

'tco3999-ng5_pc'

In [13]:
with open(f"{path_to_output}/{files[:-5]}.json", "w") as outfile:
    json.dump(out, outfile)

In [15]:
!pwd

/home/a/a270088/PYTHON/nextgems/NextGems_Cycle2/FESOM


In [14]:
xr.open_zarr(f"reference::/{path_to_output}/{files[:-5]}.json", consolidated=False)

<xarray.Dataset>
Dimensions:    (time: 985, lat: 2700, lon: 5400)
Coordinates:
  * lat        (lat) float64 -90.0 -89.93 -89.87 -89.8 ... 89.8 89.87 89.93 90.0
    latitude   (time, lat, lon) float64 dask.array<chunksize=(1, 2700, 5400), meta=np.ndarray>
  * lon        (lon) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
    longitude  (time, lat, lon) float64 dask.array<chunksize=(1, 2700, 5400), meta=np.ndarray>
  * time       (time) datetime64[ns] 2020-01-20 ... 2020-03-01
Data variables:
    2t         (time, lat, lon) float32 dask.array<chunksize=(1, 1350, 2700), meta=np.ndarray>
Attributes:
    experiment_id:  tco3999-ng5
    interpolation:  nearest neighbour